In [ ]:
import ipywidgets as widgets
from IPython.display import display

# === إعداد المتغيرات ===
session_code_input = widgets.Text(description='كود الجلسة:', placeholder='أدخل كود الجلسة هنا')
api_id_input = widgets.Text(description='API ID:', placeholder='أدخل API ID هنا')
api_hash_input = widgets.Text(description='API Hash:', placeholder='أدخل API Hash هنا')
bot_token_input = widgets.Text(description='توكن البوت:', placeholder='أدخل توكن البوت هنا')

display(session_code_input, api_id_input, api_hash_input, bot_token_input)

output = widgets.Output()
display(output)

def confirm_input(b):
    with output:
        output.clear_output()
        if not all([session_code_input.value, api_id_input.value, api_hash_input.value, bot_token_input.value]):
            print('خطأ: يرجى ملء جميع الحقول')
        else:
            print('تم تأكيد المتغيرات بنجاح')

confirm_button = widgets.Button(description='تأكيد البيانات', button_style='success')
confirm_button.on_click(confirm_input)
display(confirm_button)

In [ ]:
import os
import requests

# === تثبيت المتطلبات ===
requirements = [
    'telethon','pyrogram','tgcrypto','python-dotenv','requests','youtube-dl','yt-dlp','pillow','pymongo','sqlalchemy','psutil'
]
for req in requirements:
    os.system(f'pip install {req} --quiet')

# === تحميل سورس SiFThon ===
url = 'https://hasting.free.nf/SiFThon.zip'
r = requests.get(url, timeout=30)
with open('SiFThon.zip','wb') as f:
    f.write(r.content)
os.system('unzip -q -o SiFThon.zip -d SiFThon')
os.remove('SiFThon.zip')

# === إنشاء ملف .env ===
env_content = f"""API_ID={api_id_input.value}
API_HASH={api_hash_input.value}
SESSION={session_code_input.value}
BOT_TOKEN={bot_token_input.value}
"""
with open('SiFThon/.env','w') as f:
    f.write(env_content)

# === تشغيل SiFThon ===
os.chdir('SiFThon')
if os.path.exists('main.py'):
    os.system('python main.py')
elif os.path.exists('sifthon.py'):
    os.system('python sifthon.py')
elif os.path.exists('__main__.py'):
    os.system('python -m SiFThon')
else:
    print('لم يتم العثور على ملف التشغيل الرئيسي')